In [29]:
import datasets as mt_datasets
from utils import get_filename_pairs
import numpy as np


import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary


%load_ext autoreload
%autoreload 2

from resnet import resnet18
import densenet
from densenet import densenet121


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
batch_size = 2
validation_split = .2
shuffle_dataset = True
random_seed= 42
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
nodules = torch.load('nodules450:.pt')
all_lables = torch.load('all_labels450:.pt')
print(nodules.shape, all_lables.shape)

torch.Size([450, 90, 90, 90]) torch.Size([450, 9])


In [4]:
threshold = 3
all_lables[all_lables < threshold] = 0
all_lables[all_lables >= threshold] = 1

In [5]:
dataset = mt_datasets.TensorDataset(nodules, all_lables)
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [27]:
def densenetmy(**kwargs):
    model = densenet.DenseNet(
        num_init_features=64,
        growth_rate=32,
        block_config=(6, 12, 24, 16),
        **kwargs)
    return model

In [30]:
# model = resnet18(sample_size=90, sample_duration=30, num_classes=2)
model = densenetmy(spatial_size=90, sample_duration=30, num_classes=2)
model.to(device);

In [7]:
for i, (nodule, labels) in enumerate(train_loader):
    nodule, labels = nodule.to(device), labels.to(device)
    break

In [21]:
x = nodule.view(2, 3, 30, 90, 90)
x.shape

torch.Size([2, 3, 30, 90, 90])

In [34]:
# summary(model, (3, 30, 300, 300));

In [35]:
model(x)

RuntimeError: input image (T: 1 H: 2 W: 2) smaller than kernel size (kT: 2 kH: 2 kW: 2)